In [ ]:
import os
import cv2
from pathlib import Path
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import initializers
import matplotlib.pyplot as plt

In [ ]:
isInColab = True

In [ ]:
if isInColab:
  from google.colab import drive
  drive.mount('/content/gdrive')
  !ls -l "/content/gdrive/My Drive/FoodSeg103"
  data_folder_path =os.listdir(Path(r"/content/gdrive/My Drive/FoodSeg103/Images"))
else:
  data_folder_path =os.listdir(Path(os.getcwd()+ r"/Dataset/FoodSeg103/Images"))

In [18]:
##Constants
IMAGE_SIZE = 256
BATCH_SIZE = 8
NUM_CLASSES = 104
NUM_TRAIN_IMAGES = 128
NUM_VAL_IMAGES = 32

In [19]:
if isInColab: #Should be refactored to use data_folder_path as prefix
  train_images = os.listdir(Path(os.getcwd() + r"/gdrive/My Drive/FoodSeg103/Images/img_dir/train"))[:NUM_TRAIN_IMAGES]
  train_masks = os.listdir(Path(os.getcwd()+ r"/gdrive/My Drive/FoodSeg103/Images/ann_dir/train"))[:NUM_TRAIN_IMAGES]
  val_images = os.listdir(Path(os.getcwd()+ r"/gdrive/My Drive/FoodSeg103/Images/img_dir/train"))[NUM_TRAIN_IMAGES : NUM_VAL_IMAGES + NUM_TRAIN_IMAGES] #USE MORE DATA? ALSO TEST AND TRAIN
  val_masks = os.listdir(Path(os.getcwd()+ r"/gdrive/My Drive/FoodSeg103/Images/ann_dir/train"))[NUM_TRAIN_IMAGES : NUM_VAL_IMAGES + NUM_TRAIN_IMAGES]

  train_images = [str(Path(os.getcwd() + '/gdrive/My Drive/FoodSeg103/Images/img_dir/train', img)) for img in train_images]
  train_masks = [str(Path(os.getcwd() + '/gdrive/My Drive/FoodSeg103/Images/ann_dir/train', img)) for img in train_masks]
  val_images = [str(Path(os.getcwd() + '/gdrive/My Drive/FoodSeg103/Images/img_dir/train', img)) for img in val_images]
  val_masks = [str(Path(os.getcwd() + '/gdrive/My Drive/FoodSeg103/Images/ann_dir/train', img)) for img in val_masks]
else:
  train_images = os.listdir(Path(os.getcwd()+ r"/Dataset/FoodSeg103/Images/img_dir/train"))[:NUM_TRAIN_IMAGES]
  train_masks = os.listdir(Path(os.getcwd()+ r"/Dataset/FoodSeg103/Images/ann_dir/train"))[:NUM_TRAIN_IMAGES]
  val_images = os.listdir(Path(os.getcwd()+ r"/Dataset/FoodSeg103/Images/img_dir/train"))[NUM_TRAIN_IMAGES : NUM_VAL_IMAGES + NUM_TRAIN_IMAGES]
  val_masks = os.listdir(Path(os.getcwd()+ r"/Dataset/FoodSeg103/Images/ann_dir/train"))[NUM_TRAIN_IMAGES : NUM_VAL_IMAGES + NUM_TRAIN_IMAGES]

  train_images = [str(Path(os.getcwd() + '/Dataset/FoodSeg103/Images/img_dir/train', img)) for img in train_images]
  train_masks = [str(Path(os.getcwd() + '/Dataset/FoodSeg103/Images/ann_dir/train', img)) for img in train_masks]
  val_images = [str(Path(os.getcwd() + '/Dataset/FoodSeg103/Images/img_dir/train', img)) for img in val_images]
  val_masks = [str(Path(os.getcwd() + '/Dataset/FoodSeg103/Images/ann_dir/train', img)) for img in val_masks]


In [20]:
for ans_path in train_masks:
  print(ans_path)

/content/gdrive/My Drive/FoodSeg103/Images/ann_dir/train/00003975.png
/content/gdrive/My Drive/FoodSeg103/Images/ann_dir/train/00003974.png
/content/gdrive/My Drive/FoodSeg103/Images/ann_dir/train/00003989.png
/content/gdrive/My Drive/FoodSeg103/Images/ann_dir/train/00003994.png
/content/gdrive/My Drive/FoodSeg103/Images/ann_dir/train/00004009.png
/content/gdrive/My Drive/FoodSeg103/Images/ann_dir/train/00004002.png
/content/gdrive/My Drive/FoodSeg103/Images/ann_dir/train/00004007.png
/content/gdrive/My Drive/FoodSeg103/Images/ann_dir/train/00003995.png
/content/gdrive/My Drive/FoodSeg103/Images/ann_dir/train/00004014.png
/content/gdrive/My Drive/FoodSeg103/Images/ann_dir/train/00003997.png
/content/gdrive/My Drive/FoodSeg103/Images/ann_dir/train/00003985.png
/content/gdrive/My Drive/FoodSeg103/Images/ann_dir/train/00003988.png
/content/gdrive/My Drive/FoodSeg103/Images/ann_dir/train/00003987.png
/content/gdrive/My Drive/FoodSeg103/Images/ann_dir/train/00003970.png
/content/gdrive/My D

In [21]:
def read_image(image_path, mask=False):
    image = tf.io.read_file(image_path)
    if mask:
        image = tf.image.decode_png(image, channels=1)
        image.set_shape([None, None, 1])
        image = tf.image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE])
    else:
        image = tf.image.decode_png(image, channels=3)
        image.set_shape([None, None, 3])
        image = tf.image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE])
        image = tf.keras.applications.resnet50.preprocess_input(image)
    return image


def load_data(image_list, mask_list):
    image = read_image(image_list)
    mask = read_image(mask_list, mask=True)
    return image, mask


def data_generator(image_list, mask_list):
    dataset = tf.data.Dataset.from_tensor_slices((image_list, mask_list))
    dataset = dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    return dataset

In [22]:
train_dataset = data_generator(train_images, train_masks)
val_dataset = data_generator(val_images, val_masks)

#One batch of training images, training answers, validation images and validation answers
print("Train Dataset:", train_dataset)
print("Val Dataset:", val_dataset)

Train Dataset: <_BatchDataset element_spec=(TensorSpec(shape=(8, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(8, 256, 256, 1), dtype=tf.float32, name=None))>
Val Dataset: <_BatchDataset element_spec=(TensorSpec(shape=(8, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(8, 256, 256, 1), dtype=tf.float32, name=None))>


In [23]:
def convolution_block(
    block_input,
    num_filters=256,
    kernel_size=3,
    dilation_rate=1,
    padding="same",
    use_bias=False,
):
    x = layers.Conv2D(
        num_filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding="same",
        use_bias=use_bias,
        kernel_initializer=keras.initializers.HeNormal(),
    )(block_input)
    x = layers.BatchNormalization()(x)
    return tf.nn.relu(x)


def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape
    x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size=1, use_bias=True)
    out_pool = layers.UpSampling2D(
        size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]), interpolation="bilinear",
    )(x)

    out_1 = convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
    out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
    out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=18)

    x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    output = convolution_block(x, kernel_size=1)
    return output

In [24]:
def DeeplabV3Plus(image_size, num_classes):
    model_input = keras.Input(shape=(image_size, image_size, 3))
    resnet50 = keras.applications.ResNet50(
        weights="imagenet", include_top=False, input_tensor=model_input
    )
    x = resnet50.get_layer("conv4_block6_2_relu").output
    x = DilatedSpatialPyramidPooling(x)

    input_a = layers.UpSampling2D(
        size=(image_size // 4 // x.shape[1], image_size // 4 // x.shape[2]),
        interpolation="bilinear",
    )(x)
    input_b = resnet50.get_layer("conv2_block3_2_relu").output
    input_b = convolution_block(input_b, num_filters=48, kernel_size=1)

    x = layers.Concatenate(axis=-1)([input_a, input_b])
    x = convolution_block(x)
    x = convolution_block(x)
    x = layers.UpSampling2D(
        size=(image_size // x.shape[1], image_size // x.shape[2]),
        interpolation="bilinear",
    )(x)
    model_output = layers.Conv2D(num_classes, kernel_size=(1, 1), padding="same", kernel_initializer=keras.initializers.HeNormal())(x)
    return keras.Model(inputs=model_input, outputs=model_output)


model = DeeplabV3Plus(image_size=IMAGE_SIZE, num_classes=NUM_CLASSES)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 262, 262, 3)          0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 128, 128, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 128, 128, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                        

In [25]:
print(tf.config.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [26]:
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.002),
    loss=loss,
    metrics=["accuracy"],
)
history = model.fit(train_dataset, validation_data=val_dataset, epochs=8)

plt.plot(history.history["loss"])
plt.title("Training Loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.show()

plt.plot(history.history["accuracy"])
plt.title("Training Accuracy")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.show()

plt.plot(history.history["val_loss"])
plt.title("Validation Loss")
plt.ylabel("val_loss")
plt.xlabel("epoch")
plt.show()

plt.plot(history.history["val_accuracy"])
plt.title("Validation Accuracy")
plt.ylabel("val_accuracy")
plt.xlabel("epoch")
plt.show()

Epoch 1/8
16/16 [==============================] - 408s 23s/step - loss: 3.5520 - accuracy: 0.3395 - val_loss: 7712.0107 - val_accuracy: 0.1962
Epoch 2/8
16/16 [==============================] - 361s 23s/step - loss: 2.7324 - accuracy: 0.4550 - val_loss: 167400.9688 - val_accuracy: 0.3539
Epoch 3/8
16/16 [==============================] - 397s 25s/step - loss: 2.6120 - accuracy: 0.4569 - val_loss: 2783.8188 - val_accuracy: 0.4278
Epoch 4/8
16/16 [==============================] - 373s 24s/step - loss: 2.5408 - accuracy: 0.4545 - val_loss: 1444.7690 - val_accuracy: 0.3882
Epoch 5/8
16/16 [==============================] - 348s 22s/step - loss: 2.4776 - accuracy: 0.4599 - val_loss: 108.1597 - val_accuracy: 0.2863
Epoch 6/8
16/16 [==============================] - 376s 24s/step - loss: 2.4107 - accuracy: 0.4567 - val_loss: 110.4054 - val_accuracy: 0.2401


KeyboardInterrupt: ignored

In [ ]:
img_path = str(Path(os.getcwd() + r"/gdrive/My Drive/FoodSeg103/Images/img_dir/test/00000048.jpg"))
img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
img = cv2.resize(img, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)
plt.imshow(img);
plt.show()
ans_path = str(Path(os.getcwd() + r"/gdrive/My Drive/FoodSeg103/Images/ann_dir/test/00000048.png"))
ans = cv2.cvtColor(cv2.imread(ans_path), cv2.COLOR_BGR2RGB)
ans = cv2.resize(ans, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)
plt.imshow(ans);
plt.show()


processed_image = read_image(img_path)[None,:,:,:]
old = model.predict(processed_image)
old = old.squeeze()
new = np.zeros((IMAGE_SIZE,IMAGE_SIZE))
for i in range(IMAGE_SIZE):
  for j in range(IMAGE_SIZE):
    new[i][j] = np.argmax(old[i][j])

newAnn = np.zeros((IMAGE_SIZE,IMAGE_SIZE))
for i in range(IMAGE_SIZE):
  for j in range(IMAGE_SIZE):
    newAnn[i][j] = np.max(ans[i][j])
plt.imshow(newAnn);
plt.show()
plt.imshow(new);
plt.show()

In [ ]:
max = 0
for ans_path in train_masks:
  candidate = np.max(cv2.imread(ans_path))
  if candidate > max:
    max = candidate
    max_img = cv2.imread(ans_path)
    max_path = ans_path
print(ans_path)
print(max)
plt.imshow(max_img);
plt.show()
ans = cv2.cvtColor(max_img, cv2.COLOR_BGR2RGB)
ans = cv2.resize(ans, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)
plt.imshow(ans);
plt.show()